In [ ]:
# Import Libraries
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization, GlobalMaxPooling2D
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import scikitplot as skplt

In [ ]:
# Specify path to your data directory
path = 'Path to your data directory'

In [ ]:
# Specify image target size and batch size
target_size = (299,299)
batch_size = 32

In [ ]:
# Normalize your images and split your set into 80% training and 20% validation
datagen = ImageDataGenerator(validation_split=0.2, rescale=1./255)

In [ ]:
# Create training tensor data for your ML model
train_data_gen = datagen.flow_from_directory(
    path,
    subset='training',
    target_size=target_size, 
    batch_size=batch_size, 
    class_mode='binary')

In [ ]:
# Create validation tensor data for your model
val_data_gen = datagen.flow_from_directory(
    path,
    subset='validation',
    target_size=target_size, 
    batch_size=batch_size, 
    class_mode='binary',
    shuffle=False)

In [ ]:
# Plot sample images from your training data
class_labels = train_data_gen.class_indices
print(class_labels)

sample_training_images, labels = next(train_data_gen)

def plotImages(images_arr, labels):
    fig, axes = plt.subplots(4, 4, figsize=(15,15))
    axes = axes.flatten()
    for img, lbs, ax in zip( images_arr, labels, axes):
        ax.imshow(img)
        ax.set_title(lbs)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

plotImages(sample_training_images[:16], labels[:16])

In [ ]:
# Define your pretrained ML model (ResNet50)
input_shape = (299,299,3)

pretrained_model = ResNet50(
    input_shape = input_shape,
    weights = 'imagenet',
    layers=tf.keras.layers,
    include_top = False)

pretrained_model.trainable = False

In [ ]:
# Define your final ML model
model = Sequential()
model.add(pretrained_model)
model.add(GlobalMaxPooling2D())
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [ ]:
# Compile your model
model.compile(optimizer='Adam', loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
# Fit your model
epochs = 5

history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=train_data_gen.samples/train_data_gen.batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=val_data_gen.samples/val_data_gen.batch_size
    )

In [ ]:
# Make predictions on your validation images
predict_prob = model.predict_generator(val_data_gen)
predictions = predict_prob > 0.5

In [ ]:
# Plot confusion matrix
print(class_labels)
skplt.metrics.plot_confusion_matrix(
    val_data_gen.classes, 
    predictions,
    figsize=(10,10))